In [1]:
import sys

import numpy as np
import netCDF4 as nc
import numpy.matlib
import datetime
import xarray as xr
from scipy import interpolate
from numpy import ma
from scipy import stats
import scipy.io as sio
import pickle as pickle
from sklearn import linear_model
import numpy.ma as ma
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing

import scipy as sp
import pandas as pd

import time

from copy import copy 

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

# OS interaction
import os
import sys
import cftime

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

import matplotlib.colors as mcolors

import glob
import dask
import dask.bag as db

from scipy import interpolate

import statsmodels.stats.multitest as multitest

from matplotlib.gridspec import GridSpec
from cartopy.crs import EqualEarth, PlateCarree

In [4]:
era5_path = '/glade/campaign/collections/rda/data/ds633.0/'
out_path = '/glade/u/home/zcleveland/scratch/ERA5/dsw/'

In [18]:
var = 'swvl1'
variables = ['avg', 'min', 'max']  # List of variables

In [33]:
data_list = []

for variable in variables:
    files = glob.glob(f'{out_path}swvl1_*_{variable}.nc')
    files.sort()
    print(f'Length of files for {variable}: {len(files)}')
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ds = xr.open_mfdataset(files, concat_dim='time', combine='nested', parallel=True, chunks={'time': 'auto'})
        ds = ds.rename_vars({f'SWVL1': f'SWVL1_{variable}'})
        data_list.append(ds)

data = xr.merge(data_list, compat='override')

Length of files for avg: 40
Length of files for min: 40
Length of files for max: 40


In [34]:
data

<xarray.Dataset>
Dimensions:    (time: 14610, latitude: 81, longitude: 81)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 ... 2019-12-31
  * latitude   (latitude) float64 40.0 39.75 39.5 39.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float64 240.0 240.2 240.5 240.8 ... 259.5 259.8 260.0
Data variables:
    SWVL1_avg  (time, latitude, longitude) float32 dask.array<chunksize=(366, 81, 81), meta=np.ndarray>
    utc_date   (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    SWVL1_min  (time, latitude, longitude) float32 dask.array<chunksize=(366, 81, 81), meta=np.ndarray>
    SWVL1_max  (time, latitude, longitude) float32 dask.array<chunksize=(366, 81, 81), meta=np.ndarray>

In [17]:
print(data.isel(time=0, latitude=0, longitude=0))


<xarray.Dataset>
Dimensions:    ()
Coordinates:
    time       datetime64[ns] 1980-01-01
    latitude   float64 40.0
    longitude  float64 240.0
Data variables:
    SWVL1      float32 dask.array<chunksize=(), meta=np.ndarray>
    utc_date   int32 dask.array<chunksize=(), meta=np.ndarray>
